In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from prompt import prompt

llm = ChatOpenAI(model="gpt-4o")


# Indexing

## Load

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/manual_auditoria.pdf")
docs = loader.load_and_split()

## Split

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

## Store

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Retrive

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

custom_rag_prompt = PromptTemplate.from_template(prompt)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [9]:
question = '''
Solicito atendimento em pronto socorro para paciente de 39 anos vítima de queda da própria altura com fratura de antebraço. 
Itens na conta médica: 

1. Taxa de sala de gesso 
2. Taxa de sala de pequenos procedimentos 
3. Consulta em PS 
4. Imobilização do antebraço 
5. Honorário de sutura de pequeno ferimento
6. Honorário de anestesista 
7. Box hora (taxa de permanência em PS) 4 horas 

Forneça o código de cada item autorizado ou negado.
'''

In [10]:
response = rag_chain.invoke(question)
print(response)

# Código correspondente ao item: 
# Nome do Item: Taxa de sala de gesso
# Situação: AUTORIZADO
# Justificativa: A taxa de sala de gesso é pertinente para o procedimento de imobilização do antebraço.

# Código correspondente ao item: 
# Nome do Item: Taxa de sala de pequenos procedimentos
# Situação: RECUSADO
# Justificativa: A cobrança simultânea da taxa de sala de gesso e taxa de sala de pequenos procedimentos não é pertinente. De acordo com os documentos, devemos considerar apenas um procedimento principal.

# Código correspondente ao item: 10101039
# Nome do Item: Consulta em PS
# Situação: AUTORIZADO
# Justificativa: A consulta em pronto-socorro é pertinente para o atendimento inicial da vítima de queda.

# Código correspondente ao item: 
# Nome do Item: Imobilização do antebraço
# Situação: AUTORIZADO
# Justificativa: A imobilização do antebraço é necessária devido à fratura e está de acordo com o atendimento de urgência.

# Código correspondente ao item: 
# Nome do Item: Honorári

In [11]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Documento {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# pretty_print_docs(retriever.invoke(question))